Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

Izay misy hoe `YOUR CODE HERE` na "YOUR ANSWER HERE" ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
Eto ilay references rehetra no apetraka

---

In [ ]:
import numpy as np
import scipy
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import cvxpy as cp
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings("ignore")
from random import randrange
from sklearn.datasets import load_boston, load_diabetes, load_iris, load_digits, load_breast_cancer, make_blobs
from sklearn.model_selection import train_test_split

def grad_check_sparse(f, x, analytic_grad, num_checks=12, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Alternative Linear regression

Implement linear regression using the following alternative loss instead of MSE:

$$Loss(\mathbf{w})= \frac{1}{N} \sum_{i=1}^N h_\epsilon(\mathbf{w}^\top\mathbf{x}_i +b - \mathbf{y}_i) + \lambda ||\mathbf{w}||^2_2$$

where $$h_\epsilon(r) =  \begin{cases}
    r^2/2 & \text{if } |r|\le \epsilon \\ 
    \epsilon|r|-\epsilon^2/2 & \text{if } |r| \gt \epsilon
\end{cases}$$



In [ ]:
data = load_boston()
X_train1, y_train1 = data.data, data.target
w1 = np.random.randn(X_train1.shape[1]) * 0.0001
b1 = np.random.randn(1) * 0.0001

In [ ]:
def alternative_loss_lr_naive(w, b, X, y, epsilon=1.35, alpha=0.0001):
    """
    Naive loss for all observations
    
    Inputs:
    - w: array of shape (D,) containing weights
    - b: float bias 
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels 
    - epsilon: float
    - alpha: regularization
    """
    N = X.shape[0]
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0.0
    
    for i in range(N):
        # Calculate the prediction
        prediction = np.dot(X[i], w) + b
        # Calculate the residual
        residual = prediction - y[i]
        # Apply the piecewise function for the loss
        if abs(residual) <= epsilon:
            loss += (residual ** 2) / 2
            # Gradient update for weights and bias inside the epsilon bound
            dw += residual * X[i]
            db += residual
        else:
            loss += epsilon * abs(residual) - (epsilon ** 2) / 2
            # Gradient update for weights and bias outside the epsilon bound
            dw += epsilon * np.sign(residual) * X[i]
            db += epsilon * np.sign(residual)
            
    # Add regularization to the loss
    loss /= N
    loss += alpha * np.sum(w ** 2)
    
    # Add regularization to the gradients
    dw /= N
    dw += 2 * alpha * w
    db /= N
    
    return loss, dw, np.array(db).reshape(1,)

## without regularization

In [ ]:
loss, dw1, db1 = alternative_loss_lr_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: alternative_loss_lr_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-8)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: alternative_loss_lr_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-8)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = alternative_loss_lr_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: alternative_loss_lr_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-8)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: alternative_loss_lr_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-8)

 ## with regularization

In [ ]:
loss, dw1, db1 = alternative_loss_lr_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: alternative_loss_lr_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-8)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: alternative_loss_lr_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-8)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = alternative_loss_lr_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: alternative_loss_lr_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-8)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: alternative_loss_lr_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-8)

In [ ]:
def alternative_loss_lr_vectorized(w, b, X, y, epsilon=1.35, alpha=0.0001):
    """
    Vectorized for all observations
    
    Inputs:
    - w: array of shape (D,) containing weights
    - b: float bias 
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels 
    - epsilon: float
    - alpha: regularization
    """
    N, D = X.shape

    # Compute predictions
    predictions = X.dot(w) + b
    
    # Compute residuals
    residuals = predictions - y
    
    # Apply the piecewise function h_epsilon(r) for loss
    abs_residuals = np.abs(residuals)
    quadratic_part = np.where(abs_residuals <= epsilon, 0.5 * residuals ** 2, 0)
    linear_part = np.where(abs_residuals > epsilon, epsilon * (abs_residuals - 0.5 * epsilon), 0)
    loss = np.sum(quadratic_part + linear_part) / N
    
    # Add regularization to the loss
    loss += alpha * np.sum(w ** 2)
    
    # Compute gradient
    indicator_quadratic = abs_residuals <= epsilon
    indicator_linear = abs_residuals > epsilon
    
    dquadratic = residuals * indicator_quadratic
    dlinear = epsilon * np.sign(residuals) * indicator_linear
    
    dw = (X.T.dot(dquadratic + dlinear)) / N + 2 * alpha * w
    db = np.sum(dquadratic + dlinear) / N
    
    return loss, dw, db.reshape(1,)

## without regularization

In [ ]:
loss, dw1, db1 = alternative_loss_lr_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: alternative_loss_lr_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-8)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: alternative_loss_lr_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-8)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = alternative_loss_lr_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: alternative_loss_lr_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-8)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: alternative_loss_lr_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-8)

## with regularization

In [ ]:
loss, dw1, db1 = alternative_loss_lr_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: alternative_loss_lr_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-8)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: alternative_loss_lr_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-8)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = alternative_loss_lr_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: alternative_loss_lr_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-8)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: alternative_loss_lr_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-8)

In [ ]:
class LinearModelRegression():
    def __init__(self):
        self.w = None
        self.b = None

    def train(self, X, y, learning_rate=1e-3, alpha=0.0001, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            indices = np.random.choice(N, batch_size)
            X_batch = X[indices]
            y_batch = y[indices]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update
            # Update the weights w and bias b using the gradient and the learning rate. 
            self.w -= learning_rate * dw
            self.b -= learning_rate * db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
                
        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class AlternativeLinearRegression(LinearModelRegression):
    """ Linear regression """

    def loss(self, X_batch, y_batch, alpha):
        return alternative_loss_lr_vectorized(self.w, self.b, X_batch, y_batch, alpha=alpha)
    
    def predict(self, X):
        # Use the learned weights and bias to make predictions.
        return X.dot(self.w) + self.b

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)

model = AlternativeLinearRegression()
model.train(X_train1, y_train1, num_iters=75000, batch_size=64, learning_rate=1e-2, verbose=True)
pred = model.predict(X_train1)
mse = mean_squared_error(pred, y_train1)

print("MSE gradient descent model :", mse)
assert mse < 25

# Alternative multiclass classification

Implement classification using the following loss:

$$L(\mathbf{W}) = \sum_{i=1}^N \sum_{j \neq y_i} \max(0, s_j - s_{y_i} + 1) + \lambda||\mathbf{w}||^2_2$$
$$\text{where } s_j = (f(\mathbf{x}_i;\mathbf{W}))_j = (\mathbf{W}\mathbf{x}_i)_j \text{ is the score for the j-th class}$$

In [ ]:
data = load_iris()
X, y = data.data, data.target

W = np.random.randn(X.shape[1], 3) * 0.0001

In [ ]:
def alternative_classification_loss_naive(W, X, y, alpha):
    """
    Multiclass Naive loss function WITH FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    
    # Initialization
    loss = 0.0
    dW = np.zeros_like(W)
    N, D = X.shape
    C = W.shape[1]
    
    for i in range(N): # Iterate over all samples
        scores = X[i].dot(W)
        correct_class_score = scores[y[i]]
        
        for j in range(C): # Iterate over all classes
            if j == y[i]:
                continue # Skip the correct class to only calculate loss on incorrect classes
            margin = scores[j] - correct_class_score + 1 # Note the +1 for margin
            if margin > 0:
                loss += margin
                dW[:, j] += X[i] # Gradient update for incorrect class
                dW[:, y[i]] -= X[i] # Gradient update for correct class
    
    # Average loss over the batch and add regularization
    loss /= N
    loss += alpha * np.sum(W * W)
    
    # Average gradients over the batch and add regularization gradient
    dW /= N
    dW += 2 * alpha * W # L2 regularization gradient

    return loss, dW

In [ ]:
# NO REGLARIZATION
loss, dW = alternative_classification_loss_naive(W, X, y, 0.0)

f = lambda W: alternative_classification_loss_naive(W, X, y, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

In [ ]:
# With REGLARIZATION
loss, dW = alternative_classification_loss_naive(W, X, y, 2)

f = lambda W: alternative_classification_loss_naive(W, X, y, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

In [ ]:
def alternative_classification_loss_vectorized(W, X, y, alpha):
    """
    Multiclass vectorized loss function WITHOUT FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    N = X.shape[0]
    # Compute the scores
    scores = X.dot(W)
    
    # Compute the correct class scores
    correct_class_scores = scores[np.arange(N), y].reshape(-1, 1)
    
    # Compute margins
    margins = np.maximum(0, scores - correct_class_scores + 1)
    
    # Do not consider correct class in loss
    margins[np.arange(N), y] = 0
    
    # Compute loss
    loss = np.sum(margins) / N
    # Add regularization to the loss
    loss += alpha * np.sum(W * W)
    
    # Compute gradient
    positive_margins = (margins > 0).astype(float)
    # Count the number of classes that contribute to the loss for each sample
    positive_margins[np.arange(N), y] -= np.sum(positive_margins, axis=1)
    
    # Compute the gradient
    dW = X.T.dot(positive_margins) / N
    # Add regularization to the gradient
    dW += 2 * alpha * W

    return loss, dW

In [ ]:
# NO REGLARIZATION
loss, dW = alternative_classification_loss_vectorized(W, X, y, 0.0)

f = lambda W: alternative_classification_loss_vectorized(W, X, y, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

In [ ]:
# REGLARIZATION
loss, dW = alternative_classification_loss_vectorized(W, X, y, 2)

f = lambda W: alternative_classification_loss_vectorized(W, X, y, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

In [ ]:
class LinearModelClassification():
    def __init__(self, fit_intercept=True):
        self.W = None
        self.fit_intercept = fit_intercept

    def train(self, X, y, learning_rate=1e-3, alpha=0, num_iters=100, batch_size=200, verbose=False):
        if self.fit_intercept:
            X = np.hstack([np.ones((X.shape[0], 1)), X])
            
        N, d = X.shape
        
        C = (np.max(y) + 1) 
        if self.W is None: # Initialization
            self.W = 0.001 * np.random.randn(d, C)

        # Run stochastic gradient descent to optimize W
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            indices = np.random.choice(N, batch_size)
            X_batch = X[indices]
            y_batch = y[indices]
            
            # evaluate loss and gradient
            loss, dW = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.          
            self.W -= learning_rate * dW
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
                
        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class AlternativeClassificationModel(LinearModelClassification):
    """ Multiclass classification model """

    def loss(self, X_batch, y_batch, alpha):
        return alternative_classification_loss_vectorized(self.W, X_batch, y_batch, alpha)
    
    def predict(self, X):
        """ 
        Inputs:
        - X: array of shape (N, D) 

        Returns:
        - y_pred: 1-dimensional array of length N, each element is an integer giving the predicted class 
        """
        if self.fit_intercept:
            X = np.hstack([np.ones((X.shape[0], 1)), X])  # Add intercept term
        scores = X.dot(self.W)
        y_pred = np.argmax(scores, axis=1)
        return y_pred

In [ ]:
model = AlternativeClassificationModel()
model.train(X, y, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X)
model_accuracy = accuracy_score(y, pred)
print(model_accuracy)
assert model_accuracy > 0.97

# Logistic regression with CVXPY

In [ ]:
data2 = load_breast_cancer()
X2, y2 = data2.data, data2.target
scaler = StandardScaler()
X2 = scaler.fit_transform(X2)

In [ ]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

class LogisticRegressionCVXPY():
    def __init__(self, fit_intercept=True, alpha=1.0):
        self.w = None
        self.b = None if fit_intercept else 0
        self.fit_intercept = fit_intercept
        self.alpha = alpha
        
    def fit(self, X, y):
        N, D = X.shape
        w = cp.Variable(D)
        b = cp.Variable() if self.fit_intercept else 0
        
        # Formulate the logistic loss function
        log_likelihood = cp.sum(
            cp.multiply(y, X @ w + b) - 
            cp.logistic(X @ w + b)
        )
        
        # Add L2 regularization
        L2_regularization = (self.alpha / 9) * cp.norm(w, 2)**2
        
        # Define the objective function to maximize log likelihood
        objective = cp.Maximize(log_likelihood / N - L2_regularization)
        prob = cp.Problem(objective)
        
        # Solve the optimization problem
        prob.solve(solver=cp.ECOS, abstol=1e-10, reltol=1e-9, feastol=1e-10)
        
        # Save the learned parameters
        self.w = w.value
        if self.fit_intercept:
            self.b = b.value
        
    def predict(self, X):
        """ Return prediction labels vector of 0 or 1 """
        if self.fit_intercept and self.b is not None:
            preds = sigmoid(X @ self.w + self.b)
        else:
            preds = sigmoid(X @ self.w)
        # Convert probabilities to 0 or 1 based on threshold of 0.5
        return (preds >= 0.5).astype(int)

### without bias

In [ ]:
model = LogisticRegressionCVXPY(alpha=1e-3, fit_intercept=False)
model.fit(X2, y2)
pred = model.predict(X2)
accuracy = accuracy_score(y2, pred)
print(accuracy)
assert accuracy >= 0.98

### with bias

In [ ]:
model = LogisticRegressionCVXPY(alpha=1e-3, fit_intercept=True)
model.fit(X2, y2)
pred = model.predict(X2)
accuracy = accuracy_score(y2, pred)
accuracy = accuracy
print(accuracy)
assert accuracy >= 0.99

# Alternative Binary classification CVXPY
 
Implement a binary classification model with CVXPY whose parameters are obtained by: (label is 1 and -1 instead of 0 and 1)

$$\min_{\mathbf{w},b}\frac{1}{2}||\mathbf{w}||^2$$
$$\text{s.t } y_i(\mathbf{w}^{\top}\mathbf{x}_i + b) \ge 1, \ i=1...N$$

In [ ]:
X3, y3 = make_blobs(n_samples=300, centers=2, n_features=12, random_state=47)
scaler = StandardScaler()
X3 = scaler.fit_transform(X3)
y3[y3 == 0] = -1

In [ ]:
class BinaryClassificationModel():
    def __init__(self):
        self.w = None
        self.b = 0  # Initialized to 0, but will be optimized
    
    def fit(self, X, y):
        N, D = X.shape  # Number of samples and features
        w = cp.Variable(D)  # Weight vector
        b = cp.Variable()  # Bias term

        # Loss function: Sum of hinge losses for all samples
        # Hinge loss for binary classification: max(0, 1 - y_i * (X_i @ w + b))
        loss = cp.sum(cp.pos(1 - cp.multiply(y, X @ w + b)))
        
        # Define and solve the optimization problem
        problem = cp.Problem(cp.Minimize(loss))
        problem.solve()
        
        # Store the optimized parameters
        self.w = w.value
        self.b = b.value
        
    def predict(self, X):
        """Return the predicted label 1 or -1"""
        y_pred = X @ self.w + self.b
        return np.sign(y_pred)

In [ ]:
model = BinaryClassificationModel()
model.fit(X3, y3)
pred = model.predict(X3)
accuracy = accuracy_score(y3, pred)
print(accuracy)
assert accuracy == 1

# Alternative  Binary classification 2 CVXPY
 
Implement a binary classification model with CVXPY whose parameters minimize the loss

$$L(\mathbf{w},b) = \frac{1}{N} \sum_{i=1}^N \max(0, y_i(\mathbf{w}^{\top}\mathbf{x}_i + b)) + \lambda||\mathbf{w}||^2_2$$

In [ ]:
data4 = load_breast_cancer()
X4, y4 = data4.data, data4.target
scaler = StandardScaler()
X4 = scaler.fit_transform(X4)
y4[y4 == 0] = -1

In [ ]:
class BinaryClassificationModel2():
    def __init__(self, alpha=0):
        self.w = None
        self.b = 0  # Initialized to 0, but will be optimized
        self.alpha = alpha  # Regularization strength
    
    def fit(self, X, y):
        N, D = X.shape  # Number of samples and features
        w = cp.Variable(D)  # Weight vector
        b = cp.Variable()   # Bias term

        # Loss function: Sum of hinge losses for all samples plus L2 regularization
        hinge_loss = cp.sum(cp.pos(1 - cp.multiply(y, X @ w + b)))
        L2_reg = cp.norm(w, 2)**2
        loss = hinge_loss + self.alpha * L2_reg
        
        # Define and solve the optimization problem
        problem = cp.Problem(cp.Minimize(loss))
        problem.solve()
        
        # Store the optimized parameters
        self.w = w.value
        self.b = b.value
        
    def predict(self, X):
        """Return the predicted label 1 or -1""" 
        y_pred = X @ self.w + self.b
        return np.sign(y_pred)

In [ ]:
model = BinaryClassificationModel2(alpha=1e-3)
model.fit(X4, y4)
pred = model.predict(X4)
accuracy = accuracy_score(y4, pred)
print(accuracy)
assert accuracy >= 0.98

# Lasso with gradient descent

In [ ]:
data = load_diabetes()
X5, y5 = data.data, data.target

def mse_loss_vectorized(w, b, X, y):
    """
    MSE loss function WITHOUT FOR LOOPs , NO REGULARIZATION
    
    Returns a tuple of:
    - loss 
    - gradient with respect to weights w
    - gradient with respect to bias b
    """
    loss = 0.0
    dw = np.zeros_like(w)
    
    loss = np.mean(np.square(X @ w + b - y)) 
    dw = ((X.T @ (X @ w + b - y)) / X.shape[0])
    db =  np.sum(X @ w + b - y) / X.shape[0]
    
    return loss, dw, np.array(db).reshape(1,)

In [ ]:
def lasso_gradient_mse_loss_vectorized(w, b, X, y, alpha):
    """
    MSE loss function adding the subgradient for w
    """
    loss, dw, db = mse_loss_vectorized(w, b, X, y)
    
    # Add Lasso regularization penalty to the loss
    lasso_penalty = alpha * np.sum(np.abs(w))
    loss += lasso_penalty
    
    # Compute the subgradient for the Lasso penalty and add it to dw
    dw += alpha * np.sign(w)
    
    return loss, dw, db

In [ ]:
class LassoGradientDescent():
    def __init__(self,  alpha=0.1):
        self.w = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            indices = np.random.choice(N, batch_size)
            X_batch = X[indices]
            y_batch = y[indices]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            self.w -= learning_rate * dw
            self.b -= learning_rate * db
            
            loss_history.append(loss)
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
    
    
    def predict(self, X):
        return X @ self.w + self.b

    def loss(self, X_batch, y_batch):
        return lasso_gradient_mse_loss_vectorized(self.w, self.b, X_batch, y_batch, self.alpha)

In [ ]:
model = LassoGradientDescent(alpha=0.1)
model.train(X5, y5, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X5)
mse = mean_squared_error(pred, y5)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X5, y5)
sk_pred = sk_model.predict(X5)
sk_mse = mean_squared_error(sk_pred, y5)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

In [ ]:
model = LassoGradientDescent(alpha=2)
model.train(X5, y5, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X5)
mse = mean_squared_error(pred, y5)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X5, y5)
sk_pred = sk_model.predict(X5)
sk_mse = mean_squared_error(sk_pred, y5)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50